In [1]:
import pandas as pd

In [2]:
CATEGORICAL_COLUMNS = [
    'workclass', 'education', 'marital_status', 'occupation', 'relationship',
    'race', 'gender', 'native_country'
]


# All columns
COLUMNS = [
    'age', 'workclass', 'fnlwgt', 'education', 'education_num',
    'marital_status', 'occupation', 'relationship', 'race', 'gender',
    'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
    'income_bracket'
]

train_filename = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'  # noqa
test_filename = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test'  # noqa

# Get the train and test data
train_df = pd.read_csv(train_filename, names=COLUMNS, skipinitialspace=True)
test_df = pd.read_csv(test_filename, names=COLUMNS, skipinitialspace=True)

In [3]:
train_df.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [29]:
train_df['income_bracket'].value_counts()

income_bracket
<=50K    24720
>50K      7841
Name: count, dtype: int64

In [4]:
train_df.shape, test_df.shape

((32561, 15), (16282, 15))

In [5]:
# Get the categorical columns
categorical_train_df = train_df[CATEGORICAL_COLUMNS].copy()
categorical_test_df = test_df[CATEGORICAL_COLUMNS].copy()

# Concatenate the train and test data
categorical_train_df['is_train'] = 1
categorical_test_df['is_train'] = 0

In [7]:
train_df.columns == test_df.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

In [15]:
df = pd.concat([categorical_train_df, categorical_test_df], axis=0)
df

,workclass,education,marital_status,occupation,relationship,race,gender,native_country,is_train
0,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,United-States,1
1,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,1
2,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States,1
3,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States,1
4,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba,1
...,...,...,...,...,...,...,...,...,...
16277,Private,Bachelors,Divorced,Prof-specialty,Not-in-family,White,Female,United-States,0
16278,?,HS-grad,Widowed,?,Other-relative,Black,Male,United-States,0
16279,Private,Bachelors,Married-civ-spouse,Prof-specialty,Husband,White,Male,United-States,0
16280,Private,Bachelors,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,United-States,0


In [24]:
df = pd.get_dummies(df).astype(int)

In [28]:
df.query('is_train == 1')

,is_train,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,...,native_country_Portugal,native_country_Puerto-Rico,native_country_Scotland,native_country_South,native_country_Taiwan,native_country_Thailand,native_country_Trinadad&Tobago,native_country_United-States,native_country_Vietnam,native_country_Yugoslavia
0,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
1,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
32557,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
32558,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
32559,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [16]:
df['workclass'].value_counts()

workclass
Private             33906
Self-emp-not-inc     3862
Local-gov            3136
?                    2799
State-gov            1981
Self-emp-inc         1695
Federal-gov          1432
Without-pay            21
Never-worked           10
Name: count, dtype: int64

In [17]:
df['education'].value_counts()

education
HS-grad         15784
Some-college    10878
Bachelors        8025
Masters          2657
Assoc-voc        2061
11th             1812
Assoc-acdm       1601
10th             1389
7th-8th           955
Prof-school       834
9th               756
12th              657
Doctorate         594
5th-6th           509
1st-4th           247
Preschool          83
Name: count, dtype: int64

In [20]:
df['marital_status'].value_counts()

marital_status
Married-civ-spouse       22379
Never-married            16117
Divorced                  6633
Separated                 1530
Widowed                   1518
Married-spouse-absent      628
Married-AF-spouse           37
Name: count, dtype: int64

In [21]:
df['occupation'].value_counts()

occupation
Prof-specialty       6172
Craft-repair         6112
Exec-managerial      6086
Adm-clerical         5611
Sales                5504
Other-service        4923
Machine-op-inspct    3022
?                    2809
Transport-moving     2355
Handlers-cleaners    2072
Farming-fishing      1490
Tech-support         1446
Protective-serv       983
Priv-house-serv       242
Armed-Forces           15
Name: count, dtype: int64